INTRODUCTION

This notebook focuses on analyzing food delivery data to understand customer ordering patterns. The dataset was cleaned and explored to find meaningful trends. Basic visualizations were used to support the analysis. The goal is to extract insights that can help improve business decisions.


Loading DataSets and Import Libraries

In [1]:
# Step 1: Load CSV Data (orders)
import pandas as pd

orders = pd.read_csv("/content/orders.csv")


In [2]:
# Step 2: Load JSON Data (users)
users = pd.read_json("/content/users.json")


In [3]:
# Step 3: Load SQL Data (restaurants)
import sqlite3

conn = sqlite3.connect(":memory:")

with open("/content/restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)


In [4]:
# Step 4: Merge Orders with Users (LEFT JOIN)
orders_users = orders.merge(
    users,
    on="user_id",
    how="left"
)


In [5]:
# Step 5: Merge with Restaurants (LEFT JOIN)
final_df = orders_users.merge(
    restaurants,
    on="restaurant_id",
    how="left"
)


In [6]:
# Step 6: Rename duplicate columns (if needed)
final_df.rename(columns={
    "city_x": "user_city",
    "city_y": "restaurant_city"
}, inplace=True)


In [7]:
# Step 7: Export Final Dataset
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


Multiple Choice Questions

Which city has the highest total revenue (total_amount) from Gold members?  

In [9]:
# 1. City with highest total revenue from Gold members
# -------------------------------
gold_city_revenue = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)
print("1️⃣ Highest Gold revenue city:\n", gold_city_revenue.head(1), "\n")


1️⃣ Highest Gold revenue city:
 city
Chennai    1080909.79
Name: total_amount, dtype: float64 



Which cuisine has the highest average order value across all orders?

In [10]:
# 2. Cuisine with highest average order value
# -------------------------------
cuisine_aov = (
    df.groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)
print("2️⃣ Highest AOV cuisine:\n", cuisine_aov.head(1), "\n")


2️⃣ Highest AOV cuisine:
 cuisine
Mexican    808.021344
Name: total_amount, dtype: float64 



How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?

In [11]:
# 3. Distinct users with total spend > ₹1000
# -------------------------------
user_spend = df.groupby("user_id")["total_amount"].sum()
users_above_1000 = user_spend[user_spend > 1000].count()
print("3️⃣ Users with spend > 1000:", users_above_1000, "\n")


3️⃣ Users with spend > 1000: 2544 



Which restaurant rating range generated the highest total revenue?

In [12]:
# 4. Rating range with highest total revenue
# -------------------------------
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

df["rating_range"] = pd.cut(df["rating"], bins=bins, labels=labels)

rating_revenue = (
    df.groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)
print("4️⃣ Highest revenue rating range:\n", rating_revenue.head(1), "\n")

4️⃣ Highest revenue rating range:
 rating_range
4.6–5.0    2197030.75
Name: total_amount, dtype: float64 



/tmp/ipython-input-2016797405.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"]


Among Gold members, which city has the highest average order value?

In [13]:
# 5. Among Gold members, city with highest average order value
# -------------------------------
gold_city_aov = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)
print("5️⃣ Highest Gold AOV city:\n", gold_city_aov.head(1), "\n")


5️⃣ Highest Gold AOV city:
 city
Chennai    808.45908
Name: total_amount, dtype: float64 



Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

In [14]:
# 6. Cuisine with lowest distinct restaurants but high revenue
# -------------------------------
cuisine_stats = df.groupby("cuisine").agg(
    restaurants=("restaurant_id", "nunique"),
    revenue=("total_amount", "sum")
).sort_values(["restaurants", "revenue"], ascending=[True, False])

print("6️⃣ Lowest restaurant count but strong revenue cuisine:\n", cuisine_stats.head(1), "\n")


6️⃣ Lowest restaurant count but strong revenue cuisine:
          restaurants     revenue
cuisine                         
Chinese          120  1930504.65 



What percentage of total orders were placed by Gold members? (Rounded to nearest integer)

In [16]:
# 7. Percentage of orders by Gold members
# -------------------------------
gold_orders_pct = (
    df[df["membership"] == "Gold"].shape[0] / df.shape[0]
) * 100

print("7️⃣ % Orders by Gold members:", round(gold_orders_pct), "%\n")

7️⃣ % Orders by Gold members: 50 %



Which restaurant has the highest average order value but less than 20 total orders?

In [34]:
# find restaurant name column
possible_cols = [c for c in df.columns if "rest" in c.lower() and "id" not in c.lower()]
print("Restaurant column detected:", possible_cols)

rest_col = possible_cols[0]  # take first match

restaurant_stats = (
    df.groupby(rest_col)
      .agg(
          total_orders=("order_id", "count"),
          avg_order_value=("total_amount", "mean")
      )
      .reset_index()
)

filtered = restaurant_stats[
    restaurant_stats["total_orders"] < 20
]

result = filtered.sort_values(
    "avg_order_value",
    ascending=False
)

result.head()

Restaurant column detected: ['restaurant_name_x', 'restaurant_name_y', 'restaurant_name', 'restaurant_name_clean']


,restaurant_name_x,total_orders,avg_order_value
173,Hotel Dhaba Multicuisine,13,1040.222308
354,Sri Mess Punjabi,12,1029.180833
236,Ruchi Biryani Punjabi,16,1002.140625
343,Sri Delights Pure Veg,18,989.467222
88,Classic Kitchen Family Restaurant,19,973.167895


Which combination contributes the highest revenue?

In [23]:
# 9. Combination with highest revenue
# -------------------------------
# Filter only valid MCQ combinations
mcq_combinations = df[
    ((df["membership"] == "Gold") & (df["cuisine"].isin(["Indian", "Italian"]))) |
    ((df["membership"] == "Regular") & (df["cuisine"].isin(["Indian", "Chinese"])))
]

# Calculate revenue
combo_revenue = (
    mcq_combinations
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .reset_index()
    .sort_values("total_amount", ascending=False)
)

# Top combination (MCQ answer)
top_combo = combo_revenue.iloc[0]

print("Highest revenue combination:")
print(f"{top_combo['membership']} + {top_combo['cuisine']}")


Highest revenue combination:
Gold + Italian


During which quarter of the year is the total revenue highest?

In [33]:
# Convert order_date to datetime
df["order_date"] = pd.to_datetime(df["order_date"])

# Extract quarter number (1–4)
df["quarter"] = df["order_date"].dt.quarter

# Calculate total revenue per quarter
quarter_revenue = (
    df.groupby("quarter")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print(quarter_revenue)


quarter
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64


Numerical Answers

How many total orders were placed by users with Gold membership?

In [35]:
# 1. Total orders by Gold members
# -----------------------------------
gold_orders = df[df["membership"] == "Gold"].shape[0]
print("1️⃣ Total orders by Gold members:", gold_orders)


1️⃣ Total orders by Gold members: 4987


What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?

In [36]:
# 2. Total revenue from Hyderabad (rounded)
# -----------------------------------
hyderabad_revenue = round(
    df[df["city"] == "Hyderabad"]["total_amount"].sum()
)
print("2️⃣ Total revenue from Hyderabad:", hyderabad_revenue)

2️⃣ Total revenue from Hyderabad: 1889367


How many distinct users placed at least one order?

In [37]:
# 3. Distinct users with at least one order
# -----------------------------------
distinct_users = df["user_id"].nunique()
print("3️⃣ Distinct users:", distinct_users)

3️⃣ Distinct users: 2883


What is the average order value (rounded to 2 decimals) for Gold members?

In [38]:
# 4. Average order value for Gold members
# -----------------------------------
gold_aov = round(
    df[df["membership"] == "Gold"]["total_amount"].mean(), 2
)
print("4️⃣ Average order value (Gold):", gold_aov)

4️⃣ Average order value (Gold): 797.15


How many orders were placed for restaurants with rating ≥ 4.5?

In [39]:
# 5. Orders placed for restaurants with rating ≥ 4.5
# -----------------------------------
high_rating_orders = df[df["rating"] >= 4.5].shape[0]
print("5️⃣ Orders with rating ≥ 4.5:", high_rating_orders)

5️⃣ Orders with rating ≥ 4.5: 3374


How many orders were placed in the top revenue city among Gold members only?

In [40]:
# 6. Orders in top revenue city among Gold members
# -----------------------------------
top_gold_city = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

orders_top_gold_city = df[
    (df["membership"] == "Gold") & (df["city"] == top_gold_city)
].shape[0]

print("6️⃣ Orders in top Gold revenue city:", orders_top_gold_city)

6️⃣ Orders in top Gold revenue city: 1337


Fill in the blanks

The column used to join orders.csv and users.json is __________.

In [46]:
# 1. Column used to join orders.csv and users.json
# -----------------------------
join_column_orders_users = list(set(orders.columns) & set(users.columns))
print("1️⃣ Join column (orders & users):", join_column_orders_users)


1️⃣ Join column (orders & users): ['user_id']


The dataset containing cuisine and rating information is stored in __________ format.

In [42]:
# 2. Dataset containing cuisine and rating information format
# -----------------------------
print("2️⃣ Cuisine & rating dataset format: SQL")


2️⃣ Cuisine & rating dataset format: SQL


The total number of rows in the final merged dataset is __________.

In [43]:
# 3. Merge all datasets
# -----------------------------
merged_df = (
    orders
    .merge(users, on="user_id", how="left")
    .merge(restaurants, on="restaurant_id", how="left")
)

print("3️⃣ Total rows in final merged dataset:", merged_df.shape[0])


3️⃣ Total rows in final merged dataset: 10000


If a user has no matching record in users.json, the merged values will be __________.

In [44]:
# 4. Missing user records after merge
# -----------------------------
print("4️⃣ Missing user values appear as: NaN")


4️⃣ Missing user values appear as: NaN


The Pandas function used to combine datasets based on a key is __________.

In [45]:
# 5. Pandas function used to combine datasets
# -----------------------------
print("5️⃣ Pandas function used for joining: merge()")


5️⃣ Pandas function used for joining: merge()


The column membership in the final dataset originates from the __________ file.  

In [47]:
# 6. Source of 'membership' column
# -----------------------------
membership_source = "users.json" if "membership" in users.columns else "unknown"
print("6️⃣ 'membership' column originates from:", membership_source)


6️⃣ 'membership' column originates from: users.json


The join key used to combine orders data with restaurant details is __________.  

In [48]:
# 7. Join key for orders & restaurant details
# -----------------------------
join_column_orders_restaurants = list(set(orders.columns) & set(restaurants.columns))
print("7️⃣ Join key (orders & restaurants):", join_column_orders_restaurants)

7️⃣ Join key (orders & restaurants): ['restaurant_id', 'restaurant_name']


The column that helps identify the type of food served by a restaurant is __________.  

In [49]:
# 8. Column identifying type of food
# -----------------------------
food_type_column = "cuisine" if "cuisine" in restaurants.columns else "unknown"
print("8️⃣ Column identifying food type:", food_type_column)

8️⃣ Column identifying food type: cuisine


If a user places multiple orders, their personal details appear __________ times in the final merged dataset.  

In [50]:
# 9. Effect of multiple orders by same user
# -----------------------------
user_order_counts = final_df["user_id"].value_counts()
multiple_orders_effect = "multiple times" if user_order_counts.max() > 1 else "once"
print("9️⃣ User personal details appear:", multiple_orders_effect)

9️⃣ User personal details appear: multiple times


The analysis shows clear patterns in orders, revenue, and delivery performance. Customer behavior and demand trends were easier to understand through visualization. These insights can help improve delivery efficiency and customer satisfaction. Overall, the notebook demonstrates the importance of data-driven decision making.